# 08 - Carbon-Based Materials

**Overview** 

This notebook guides you through ...  

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
import matplotlib.pyplot as plt
!pip install -q qepy > /dev/null
from qepy.calculator import QEpyCalculator
!pip install -q ase > /dev/null
from ase.build import bulk, graphene
from ase.io import write


**Problem** 

Lorem Ipsum ... 

**Model**

Lorem Ipsum ...

>Smart question?

Lorem Ipsum ...

**Questions**

Before you run any simulation, answer the following question(s):

1. Something
2. Seomthing else

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Other questions

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
a_min = 2.0 # @param {type:"number"}
a_max = 5.0 # @param {type:"number"}
steps = 20 # @param {type:"number"}
ecut = 20  # @param {type:"number"}
conv_thr = 1e-6  # @param {type:"number"}
phase = "diamond"  # @param ["diamond", "graphite", "graphene"]

In [ ]:
# Lattice parameter for diamond-C (Angstrom)
a_dia = 3.567

# Conventional diamond cubic cell
diamond = bulk('C', 'diamond', a=a_dia)

qe_options = {
    '&control': {
        'calculation': "'scf'",
        'prefix': "'tmp'",
        'pseudo_dir': "'./'",
    },
    '&system': {
        'ibrav': 0,
        'ecutwfc': ecut,
        'ecutrho': 10*ecut,
    },
    '&electrons': {
        'conv_thr': conv_thr,
    },
    'atomic_species': ['C 28.0855 C.pbe-n-kjpaw_psl.1.0.0.UPF'],
    'k_points automatic': ['9 9 1 0 0 0'],
}

calc = QEpyCalculator(qe_options=qe_options, logfile='C_diamond.out')
diamond.calc=calc

# Run the calculation
#energy = diamond.get_potential_energy()
#efermi = calc.get_fermi_level()


In [ ]:
qe_options['k_points automatic']= ['8 8 8 0 0 0']
#energies, dos =  diamond.calc.get_dos(qe_options, width=0.6)
plt.plot(energies, dos)
plt.axvline(x=0, ls='--')

In [ ]:
lat = diamond.cell.get_bravais_lattice()
lat.plot_bz(show=True)
print(lat.description())

In [ ]:
path = diamond.cell.bandpath('GKLG',npoints=61)

In [ ]:
qe_options['&system']['nbnd']=6
band = diamond.calc.get_band_structure(qe_options, kpts=path, reference=efermi)
bp=band.plot()
plt.show()

**Homework Assignment**

Pick one (or more) of the following projects:
1. Select a periodic molecular or crystalline system and perform DFT simulations with at least two different density functionals (e.g. LDA and PBE)
2. Optimize the lattice parameter of the system (equation of state)
3. Verify the effect of the size of the basis set on the optimal lattice parameter
